# Inicializace

In [35]:
from collections import deque                # Trida pro ukladani stavu
from __future__ import division              # Deleni realnych cisel (kvuli verzi Pythonu 2.6)

import numpy as np                           # Knihovna pro matematicke operace
import random                                # Knihovna pro nahodny vyber samplu z pameti
import sys                                   # Pro pripojeni knihovny Open AI Gym
sys.path.append('/home/xbucha02/libraries')  # Pripojeni knihovny Open AI Gym
import gym                                   # Knihovna Open AI Gym
#from gym import wrappers                    # Nahravani zaznamu
env = gym.make('CartPole-v1')             # Konkretni hra z Open AI Gym
actionCount = env.action_space.n             # Pocet vstupu do prostredi
stateSize = env.observation_space.shape[0]   # Pocet vystupu z prostredi
#env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True) #Nahravani zaznamu

gpuMemoryUsage=1.2                            # Vyuziti pameti GPU
import tensorflow as tf                     # Knihovna TensorFlow
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
set_session(tf.Session(config=config))

# Parametry
observetime = 500                          # Delka pozorovani
episodes = 1500                            # Pocet epizod
games = 100                                # Pocet her
trainingAfterSucces = 100                  # Pocet trenovani na uspesnem datasetu
scores = []                                # Pole pro ulozeni vysledku na analyzu
episodesList = []                          # Pole pro ulozeni cisel epizod na analyzu
bestScore = 0                              # Promenna pro ukladani nejlepsiho prubezneho vysledku

# Agent

In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics

class Agent:
    def __init__(self, env):
        self.startEpsilon = 1                           # Pravdepodobnost konani nahodneho tahu na zacatku
        self.endEpsilon = 0.01                          # Pravdepodobnost konani nahodneho tahu na konci
        self.currentEpsilon = self.startEpsilon         # Soucasna pravdepodobnost konani nahodneho tahu
        self.epsilonDiminution = 0.995                  # Hodnota snizovani epsilonu
        self.gamma = 0.95                               # Discount faktor
        self.minibatchSize = 32                         # Velikost minibatche
        self.actionCount = env.action_space.n           # Pocet vstupu do prostredi
        self.stateSize = env.observation_space.shape[0] # Pocet vystupu z prostredi
        self.learningRate = 0.005                       # Learning rate
        self.fractionUpdate = 0.125
        self.memorySize = 2000                          # Velikost Replay memory
        self.primaryMemory = deque(maxlen=self.memorySize)
        self.secondaryMemory = deque(maxlen=self.memorySize)
        
        self.net = self.getNN(env)
        self.netTarget = self.getNN(env)
        self.updateTargetNet()

    def getNN(self, env):
        net = Sequential()
        net.add(Dense(24, activation="relu", input_shape=(2,) + env.observation_space.shape ))
        net.add(Flatten())
        net.add(Dense(24, activation="relu"))
        
        net.add(Dense(self.actionCount, activation="linear"))

        net.summary()

        net.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(lr=self.learningRate), metrics=['accuracy'])

        return net
    
    def updateTargetNet(self):
        self.netTarget.set_weights(self.net.get_weights())
    
    def updateTargetNetPartially(self):
        weights = self.net.get_weights()
        weightsTarget = self.netTarget.get_weights()
        
        for i in range(len(weightsTarget)):
            weightsTarget[i] = weights[i] * self.fractionUpdate + weightsTarget[i] * (1 - self.fractionUpdate)
            
        self.netTarget.set_weights(weightsTarget)
        
    def rememberPrimMem(self, state, action, reward, nextState, done):
        self.primaryMemory.append((state, action, reward, nextState, done))
    
    def rememberSecMem(self, state, action, reward, nextState, done):
        self.secondaryMemory.append((state, action, reward, nextState, done))
        
    def clearSecMem(self):
        self.secondaryMemory.clear()
        
    def epsilonActulization(self):
        if self.currentEpsilon > self.endEpsilon:
            if (self.currentEpsilon * self.epsilonDiminution) > self.endEpsilon:
                self.currentEpsilon = self.currentEpsilon * self.epsilonDiminution
            else:
                self.currentEpsilon = self.endEpsilon
    
    def getActionWE(self, state):
        if np.random.rand() <= self.currentEpsilon:
            return np.random.randint(0, self.actionCount, size=1)[0]
        else:
            Q = self.net.predict(state)
            return np.argmax(Q)

    def getAction(self, state):
        Q = self.net.predict(state)
        return np.argmax(Q)
    
    def resetSecMem(self):
        self.secondaryMemory = deque(maxlen=self.memorySize)
        
    def resetEpsilon(self):
        self.currentEpsilon = self.startEpsilon
    
    def trainDQN(self, typeOfMem):
        if typeOfMem == 1:
            if len(self.primaryMemory) >= self.minibatchSize:
                minibatch = random.sample(self.primaryMemory, self.minibatchSize)
            else:
                return
        elif typeOfMem == 2:
            if len(self.secondaryMemory) >= self.minibatchSize:
                minibatch = random.sample(self.secondaryMemory, self.minibatchSize)
            else:
                return
        else:
            return

        for i in range(0, self.minibatchSize):
            state = minibatch[i][0]
            action = minibatch[i][1]
            reward = minibatch[i][2]
            nextState = minibatch[i][3]
            done = minibatch[i][4]

            target_f = self.net.predict(state)

            if done:
                target_f[0][action] = reward
            else:
                aNet = self.net.predict(nextState)[0]

                target_f[0][action] = reward + self.gamma * np.max(aNet)

            self.net.fit(state, target_f, epochs=1, verbose=0)
                
    def trainDDQN(self, typeOfMem):
        if typeOfMem == 1:
            if len(self.primaryMemory) >= self.minibatchSize:
                minibatch = random.sample(self.primaryMemory, self.minibatchSize) #z D vybere pocet mb_size samplu
            else:
                return
        elif typeOfMem == 2:
            if len(self.secondaryMemory) >= self.minibatchSize:
                minibatch = random.sample(self.secondaryMemory, self.minibatchSize) #z D vybere pocet mb_size samplu
            else:
                return
        else:
            return
        
        #states = np.empty(self.minibatchSize)
        #print("{}" .format(states))
        #target_fs = np.empty(self.minibatchSize)
        states = np.vstack([i[0] for i in minibatch])
        target_fs = []
        
        
        for i in range(0, self.minibatchSize):
            state = minibatch[i][0]
            action = minibatch[i][1]
            reward = minibatch[i][2]
            nextState = minibatch[i][3]
            done = minibatch[i][4]
            
            target_f = self.net.predict(state)

            if done:
                target_f[0][action] = reward
            else:
                aNet = self.net.predict(nextState)[0]
                tNet = self.netTarget.predict(nextState)[0]
                target_f[0][action] = reward + self.gamma * tNet[np.argmax(aNet)]
            print("{}".format(state))
            print("{}".format(target_f))
            #states.append(state)
            target_fs.append(target_f)
        #self.net.fit(states, target_fs, batch_size=32, epochs=1, verbose=0)
        self.net.train_on_batch(states, target_fs)
    
    def loadNN(self, name):
        self.net.load_weights(name)

    def saveNN(self, name):
        self.net.save_weights(name)

# Algoritmus

In [37]:
import copy
agent = Agent(env) #vytvoreni agenta
    
#agent.loadNN("./DDQN-MountainCar-v0.h5") #nacteni NN
#agent.updateTargetNet() #nacteni NN do netTarget

for eps in range (episodes):
    state = env.reset() #resetovani prostredi
    state = np.reshape(state, [1, stateSize]) #formatovani

    state1 = copy.copy(state)
    #state2 = copy.copy(state)
    #state3 = copy.copy(state)
    
    state = np.concatenate((state, state1))
    #state = np.concatenate((state, state2))
    #state = np.concatenate((state, state3))

    state = np.expand_dims(state, axis=0)
    
    agent.epsilonActulization() #aktualizace epsilon
    
    #env.render()

    for t in range(observetime):

        action = agent.getActionWE(state) #ziskani akce

        newState, reward, done, info = env.step(action) #provedeni akce
        
        nextState = np.reshape(newState, [1, stateSize]) #formatovani
        
        temp = copy.copy(nextState)
        
        nextState = np.concatenate((nextState, state1))
        #nextState = np.concatenate((nextState, state2))
        #nextState = np.concatenate((nextState, state3))
        
        #state3 = copy.copy(state2)
        #state2 = copy.copy(state1)
        state1 = copy.copy(temp)
        
        nextState = np.expand_dims(nextState, axis=0)
        #print ("{}".format(nextState))
        agent.rememberPrimMem(state, action, reward, nextState, done) #ulozeni do primarni pameti
        agent.rememberPrimMem(state, action, reward, nextState, done) #ulozeni do sekundarni pameti
        
        #agent.trainDDQN(1) #trenovani na primarni pameti
        
        state = nextState #zmena stavu
        
        if done: #konec epizody
            print("Episode: {}/{}, epsilon: {:.2}, score: {}".format(eps, episodes, agent.currentEpsilon, t))
            agent.trainDDQN(1)
            scores.append(t) #ulozeni aktualniho skore
            episodesList.append(eps) #ulozeni aktualniho cisla epizody
            
            agent.updateTargetNet() #aktualizace target site
            
            #if t > bestScore: #bylo dosazeno njelepsiho skore
            #    bestScore = t #nove nejlepsi skore
            #    for i in range(trainingAfterSucces): #pocet trenovani
            #        agent.trainDDQN(2) #trenovani na sekundarni pameti
            #    agent.saveNN("./DDQN-CartPole-v0-{}.h5" .format(eps)) #ulozeni site
            #    #agent.resetEpsilon() #resetovani epsilonu
            
            #agent.resetSecMem() #vycisteni sekundarni pameti
            
            break
            
agent.saveNN("./DDQN-CartPole-v0.h5") #ulozeni site

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 2, 24)             120       
_________________________________________________________________
flatten_23 (Flatten)         (None, 48)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 24)                1176      
_________________________________________________________________
dense_69 (Dense)             (None, 2)                 50        
Total params: 1,346.0
Trainable params: 1,346.0
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 2, 24)             120       
_________________________________________________________________
fl

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 arrays but instead got the following list of 32 arrays: [array([[ 1.15163529,  0.10038289]], dtype=float32), array([[-0.17746359,  1.07226169]], dtype=float32), array([[ 1.08932066,  0.08144511]], dtype=float32), array([[ 1.04438627, -0.07377844]], dtype=f...

# Výsledky

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt
import copy

def analysis(scores, episodesList):
    score1 = copy.copy(scores)
    score2 = copy.copy(scores)
    score3 = copy.copy(scores)

    for i in range (len(scores)):
        if i > 1 and i < (len(scores)-2):
            score1[i] = (scores[i - 2] + scores[i - 1] + scores[i] + scores[i + 1] + scores[i + 2])/5

    for i in range (len(scores)):
        if i > 4 and i < (len(scores)-5):
            score2[i] += scores[i - 5] + scores[i - 4] + scores[i - 3] + scores[i - 2] + scores[i - 1]
            score2[i] += scores[i + 5] + scores[i + 4] + scores[i + 3] + scores[i + 2] + scores[i + 1]
            score2[i] = score2[i]/11

    for i in range (len(scores)):
        if i > 49 and i < (len(scores) - 50):
            for e in range (1,50):
                score3[i] += scores[i - e] + scores[i + e] 
            score3[i] = score3[i]/101      

    plt.plot(episodesList, scores, 'ro')
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Vysledky")
    plt.show()

    plt.plot(scores)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku")
    plt.show()

    plt.plot(score1)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku (filtr - prumer 5-ti prvku)")
    plt.show()

    plt.plot(score2)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku  (filtr - prumer 11-cti prvku)")
    plt.show()

    plt.plot(score3)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku  (filtr - prumer 101 prvku)")
    plt.show()
    
analysis(scores, episodesList)

# Hra

In [ ]:
agent.loadNN("./DDQN-CartPole-v0.h5")
observation = env.reset()
scores = []                                # Vycisteni pole pro ulozeni vysledku na analyzu
episodesList = []                          # Vycisteni pole pro ulozeni cisel epizod na analyzu

for g in range (games):
    state = env.reset()
    state = np.reshape(state, [1, stateSize])
    
    state1 = copy.copy(state)
    state2 = copy.copy(state)
    state3 = copy.copy(state)
    
    state = np.concatenate((state, state1))
    state = np.concatenate((state, state2))
    state = np.concatenate((state, state3))

    state = np.expand_dims(state, axis=0)
    
    totalReward = 0
    done = False
    
    while not done:
        #env.render()
        
        action = agent.getAction(state)
        newState, reward, done, info = env.step(action)

        nextState = np.reshape(newState, [1, stateSize])

        temp = copy.copy(nextState)
        
        nextState = np.concatenate((nextState, state1))
        nextState = np.concatenate((nextState, state2))
        nextState = np.concatenate((nextState, state3))
        
        state3 = copy.copy(state2)
        state2 = copy.copy(state1)
        state1 = copy.copy(temp)
        
        nextState = np.expand_dims(nextState, axis=0)
        
        state = nextState

        totalReward += reward
        
    scores.append(totalReward) #ulozeni aktualniho skore
    episodesList.append(g) #ulozeni aktualniho cisla epizody

    print('Game {}/{}, score: {}'.format(g, games, totalReward))

analysis(scores, episodesList)